# Protfolio optimization with CIM

## Equations

### Optimization problem formulation

Potential:
$$
V(\omega; \mu, \Theta) = \sum_{ij}^N -\mu_i \omega_i + \frac{\gamma}{2} \Theta_{ij} \omega_i \omega_j 
$$
Constraints:


-Allocations (Box constraints)
$$
\sum_{i \in I_k} \omega_i \leq \lambda_{I_k}
$$
for some subsets $I_k \subseteq \left[ N \right]$, whre $k \in [M']$ 

-Budget constraints 
$$
\sum_{i \in J_k} \omega_i = \lambda_{J_k}
$$
for some subsets $J_k \subseteq \left[ N \right]$, whre $k \in [M]$ , with $\lambda_{\left[ N \right]} = \lambda_0 = 1$

-Continuous formulation of budget constraints (with lagrange multiplier $\rho$)
$$
U(\omega; u, \lambda) = -\rho \sum_{k=0}^M \sum_i \left[ (u_k)_i \omega_i - \lambda_k \right]^2
$$
where $(u_k)_i = 1$ if the $i^{\text{th}}$ asset is in $J_k$, $0$ otherwise.

-Trading costs (backwards, real-life)
$$
\underset{\omega_i(t)}{\text{min}} \underbrace{\sum_i^N \left| \omega_i (t) - \overbrace{\omega_i(t - \delta t)}^{\equiv \Omega_i} \right| \nu_i }_{ \equiv W(\omega; \nu, \Omega) }
$$
-Trading costs (modified to have continuous derivative)
$$
\underset{\omega_i(t)}{\text{min}} \underbrace{\sum_i^N \left( \omega_i  - \Omega_i \right)^2\nu_i }_{ \equiv W'(\omega; \nu, \Omega) }
$$
-Trading cost, each transaction has a fix additional cost

### Gradient rule
$$
\frac{d \omega_k}{dt} = -\frac{\partial_k }{\partial \omega_k } \left[ V(\omega; \mu, \Theta) + W'(\omega; \nu, \Omega) + U(\omega; u, \lambda)\right]
$$

Projection to enforce box constraints...


$$
\begin{aligned}

- \frac{\partial }{\partial \omega_k }  V(\omega; \mu, \Theta) &= - \partial_k \left( \sum_{ij}^N -\mu_i \omega_i + \frac{\gamma}{2} \Theta_{ij} \omega_i \omega_j  \right) \\

 &= - \sum_{ij}^N -\mu_i \delta_{i,k} + \frac{\gamma}{2} \Theta_{ij} \delta_{i,k} \omega_j + \frac{\gamma}{2} \Theta_{ij} \omega_i \delta_{k, j}  \\

 &= \sum_{i}^N \mu_k  - \gamma \Theta_{ik} \omega_i \\

 &= \mu_k - \gamma \sum_{i}^N \Theta_{ik} \omega_i \\
\end{aligned}
$$

$$

\begin{aligned}
-\frac{\partial }{\partial \omega_k } W'(\omega; \nu, \Omega) &= - \partial_k \sigma \sum_i (\omega_i - \Omega_i)^2 \nu_i\\

&= - 2 \sigma  \sum_i \nu_i \left(\omega_i -\Omega_i \right)\delta_{i,k} \\

&= - 2 \sigma  \nu_k \left(\omega_k -\Omega_k \right)
\end{aligned}
$$



$$
\begin{aligned}
-\frac{\partial }{\partial \omega_k } U(\omega, u, \lambda) 
 &= \partial_k \rho \sum_{j=0}^M \sum_i \left[ (u_j)_i \omega_i - \lambda_j \right]^2 \\
 &= \sum_{j=0}^M \sum_i^N2 \left((u_j)_i \omega_i - \lambda_j \right) (u_j)_i \delta_{ik} \\
 &= 2 \sum_{j=0}^M \left((u_j)_k \omega_k - \lambda_j \right) (u_j)_k
\end{aligned}
$$

### Exact solution
$$
\frac{\partial}{\partial \omega_k} (V + W + U) = 0
$$
In vector notation
$$
 0 = \vec{\mu} - \gamma \underline{\underline{\Theta}} \vec{\omega} -2 \sigma \cdot \text{diag}(\vec{\nu})\left( \vec{\omega} - \vec{\Omega} \right) + \rho \sum_j \underbrace{\vec{u}_j \odot \vec{\omega}  \odot \vec{u}_j}_{\text{diag}(\vec{u_j})^2 \vec{\omega}} - \lambda_j \vec{u}_j 
$$

$$
\vec{\omega} = \left( 2\rho \sum_j \text{diag}(\vec{u_j})^2 - \gamma \underline{\underline{\Theta}} -2 \sigma \cdot \text{diag}(\vec{\nu}) \right)^{-1} \left(2 \rho \sum_j \lambda_j \vec{u}_j + 2\sigma \text{diag}(\vec{\nu}) \vec{\Omega} - \vec{\mu}  \right)
$$

## Binary encoding

$$
\left\{1, 2, \dots, M \right\} \ni \omega_i = \sum_{q=0}^{[\log_2 M]} 2^{q-2}(s_q+1)
$$
or inversly
$$
s_j = 1-2\text{Heavyside} \left[ \omega \cdot 2^{1-j} -1 \right]
$$

$$
H(\omega) = \sum_{ij} J_{ij} \omega_i \omega_j = \sum_{ij} J_{ij} \left( \sum_{p}2^{p-2} (s_p+1) \right) \left(\sum_{q}2^{q-2} (s_q+1) \right)
$$

$$
\frac{\partial}{\partial s_k} H\left( \omega(s) \right) = \frac{\partial H}{\partial \omega_l}\frac{\partial \omega_l}{\partial s_k} = \left(\sum_i J_{il} \omega_i \right) \left( \partial_k \sum_{q=0}^{[\log_2 M]} 2^{q-2}(s_q+1) \right)
$$

$$
\partial_k H = \sum_i J_{il} \omega_i 2^{k-2} = \sum_i J_{il} 2^{k-2} \left(\sum_{q}2^{q-2} (s_q+1) \right)
$$

if $s_k$ is part of the representation of $\omega_l$, otherwise it is zero.

$\pm$

# Code

In [ ]:
using IJulia
using DifferentialEquations
using DynamicalSystems
using PlotlyJS#, DataFrames
using SparseArrays
using Random
using LinearAlgebra
#using StatsBase
# using Polynomials, SpecialPolynomials
using JSON

### Depricated functions

In [ ]:
function get_ω_clipped(s, state_sizes, values, N, M)
    ω = zeros(N)
    s_clipped = [elem < 0 ? -1 : 1 for elem in s]
    idx = 0
    for i in 1:N
        ω_idx = 1 + Int(round( sum([2.0^(q-2)*(s_clipped[q+idx] + 1.0) for q in 1:state_sizes[i]]) ))
        if ω_idx > length(values[i])[1] # This has to be decided (maybe put random values from the allowed set)
            ω = 0
        else
            ω[i] = values[i][ω_idx] # ennek nincs értelem a második index 1:M_i' ig fut
        end
        idx += state_sizes[i]
    end
    return ω
end

function g_ω_old(ω, points, ϵ)
    N = size(ω)[1]
    result = zero(ω)
    for i in 1:N
        values = points[i]
        if ω[i] <= (values[1] - ϵ/2)
            result[i] = (ω[i] - (values[1] - ϵ/2))^2
        elseif ω[i] >= (last(values) + ϵ/2)
            result[i] = (ω[i] - (last(values) + ϵ/2))^2
        else
            for j in 1:(size(values)[1])
                if ω[i] > (values[j] - ϵ/2) && ω[i] < (values[j] + ϵ/2)
                    result[i] = -sqrt(0.25ϵ^2 -(ω[i] - values[j])^2)
                    break
                elseif ω[i] > (values[j] + ϵ/2) && ω[i] < (values[j+1] - ϵ/2)
                    result[i] = sqrt(0.25*(values[j+1] - values[j] - ϵ)^2 -(ω[i] - 0.5*(values[j]+values[j+1]))^2)
                    break
                else
                    result[i] = 0
                end
            end
        end
    end
    return result
end

function special_sawtooth(x, values, amplitude)
    N = size(x)[1]
    result = zero(x)

    for i in 1:N
        points = values[i]
        num_teeth = length(points) - 1
        if x[i] < points[1]
            result[i] = x[i] * amplitude / points[1]  # Linearly increasing function
        elseif x[i] > points[end]
            last_tooth_width = points[end] - points[end - 1]
            relative_x = x[i] - points[end]
            result[i] = (relative_x / last_tooth_width - 1) * amplitude  # Linearly decreasing function
        else
            for j in 1:num_teeth
                if x[i] >= points[j] && x[i] <= points[j + 1]
                    tooth_width = points[j + 1] - points[j]
                    relative_x = x[i] - points[j]
                    tooth_position = relative_x / tooth_width
                    result[i] = (2 * tooth_position - 1) * amplitude
                end
            end
        end
    end
    return result
end

# Half circles
function f_ω(ω, values, ϵ)
    if ω <= (values[1] - ϵ/2)
        return (ω - (values[1] - ϵ/2))^2
    elseif ω >= (last(values) + ϵ/2)
        return (ω - (last(values) + ϵ/2))^2
    else
        for j in 1:(size(values)[1])
            if ω > (values[j] - ϵ/2) && ω < (values[j] + ϵ/2)
                return -sqrt(0.25ϵ^2 -(ω - values[j])^2)
            elseif ω > (values[j] + ϵ/2) && ω < (values[j+1] - ϵ/2)
                return sqrt(0.25*(values[j+1] - values[j] - ϵ)^2 -(ω - 0.5*(values[j]+values[j+1]))^2)
            end
        end
    end
    return 0
end

function get_Lagrange_poly(opo_nodes, peek_param)
    function generate_peeks(nodes, peek_param)
        peek_x = Vector{Float64}()
        peek_y = Vector{Float64}()
        dist = nodes[2] - nodes[1]

        push!(peek_x, maximum(nodes) + 0.5 * dist)
        push!(peek_y, dist)
        push!(peek_x, minimum(nodes) - 0.5 * dist)
        push!(peek_y, dist)

        if length(opo_nodes) % 2 == 1
            push!(peek_x, (nodes[1]+nodes[2])/2)
            push!(peek_y, dist)
            push!(peek_x, (nodes[end]+nodes[end-1])/2)
            push!(peek_y, dist)
        else
            push!(peek_x, (nodes[Integer(length(nodes)/2)] + nodes[Integer(length(nodes)/2)+1])/2 )
            push!(peek_y, dist)
        end
        return (peek_x, peek_y)
    end
    peek_x, peek_y = generate_peeks(opo_nodes, peek_param)

    xs = Vector{Float64}()
    ys = Vector{Float64}()
    
    for i in 1:length(opo_nodes)
        push!(xs, opo_nodes[i])
        push!(ys, 0.0)
    end
    
    for (p_x, p_y) in zip(peek_x, peek_y)
        push!(xs, p_x)
        push!(ys, p_y)
    end

    perm=sortperm(xs)
    xs = xs[perm]
    ys = ys[perm];

    return Lagrange(xs, ys)
end

function V_mpo_old(x::Float64, a::Float64, per::Float64, xmax::Float64)::Float64
    term1 = (sin(π * x / per)^2) / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1))
    term2 = (x * (x - xmax) * (1 - 1 / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1)))) / a
    
    return term1 + term2
end


function DV_mpo_old(x::Float64, a::Float64, per::Float64, xmax::Float64)::Float64
    term1 = (1 - 1 / ((1 + exp(-a * x)) * (1 + exp(-a * (-x + xmax))))) * x / a
    term2 = (1 - 1 / ((1 + exp(-a * x)) * (1 + exp(-a * (-x + xmax))))) * (x - xmax) / a
    term3 = ((a * exp(-a * (-x + xmax))) / ((1 + exp(-a * x)) * (1 + exp(-a * (-x + xmax)))^2) -
             (a * exp(-a * x)) / ((1 + exp(-a * x))^2 * (1 + exp(-a * (-x + xmax))))) * x * (x - xmax) / a
    term4 = (2 * π * cos(π * x / per) * sin(π * x / per)) / ((1 + exp(-a * x)) * (1 + exp(-a * (-x + xmax))) * per)
    term5 = (a * exp(-a * (-x + xmax)) * sin(π * x / per)^2) / ((1 + exp(-a * x)) * (1 + exp(-a * (-x + xmax)))^2)
    term6 = (a * exp(-a * x) * sin(π * x / per)^2) / ((1 + exp(-a * x))^2 * (1 + exp(-a * (-x + xmax))))
    
    return term1 + term2 + term3 + term4 - term5 + term6
end


### Misc. functions

In [ ]:
function get_Jacobian(state_sizes)
    N = length(state_sizes)
    M = sum(state_sizes)

    Jakobian = zeros(Float64, (N, M))
    q = 0

    for i in 1:N
        for j in 1:M
            if j <= sum(state_sizes[1:i])
                if i > 1
                    if  j > sum(state_sizes[1:i-1])
                        q = j - 1 - sum(state_sizes[1:i-1])
                        Jakobian[i, j] = 2.0^(q-2)
                    end
                else
                    q = j-1
                    Jakobian[i, j] = 2.0^(q-2)
                end
            end
        end
    end
    return sparse(Jakobian)
end

function g_ω(a, ω, set_of_zeros)
    function σωσ(a, ω, α)
        return (2/(1 + exp(-a*(ω - α))) - 1)
    end

    if ω <= 0.0
        return -1.0
    end

    product = 1.0
    if length(set_of_zeros) %2 == 1
        for i in 1:length(set_of_zeros)-1
            product *= σωσ(-a, ω, set_of_zeros[i]) * σωσ(a, ω,  0.5*(set_of_zeros[i]+set_of_zeros[i+1]))
        end
        return product * σωσ(-a, ω, set_of_zeros[end]) * (σωσ(a, ω, 0)+1)
    else
        for i in 1:length(set_of_zeros)-1
            product *= σωσ(-a, ω, set_of_zeros[i]) * σωσ(a, ω,  0.5*(set_of_zeros[i]+set_of_zeros[i+1]))
        end
        return - product * σωσ(-a, ω, set_of_zeros[end]) * (σωσ(a, ω, 0)+1)
    end
end

function DV_mpo(x::Float64, a::Float64, per::Float64, xmax::Float64)::Float64
    term1 = (a * exp(-a * x) * sin(pi * x / per)^2) / ((exp(-a * x) + 1)^2 * (exp(-a * (xmax - x)) + 1))
    term2 = (a * exp(-a * (xmax - x)) * sin(pi * x / per)^2) / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1)^2)
    term3 = (2 * pi * sin(pi * x / per) * cos(pi * x / per)) / (per * (exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1))
    term4 = a * (exp(-x) - 1) * exp(x - xmax) * (1 - 1 / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1)))
    term5 = a * exp(-x) * (exp(x - xmax) - 1) * (1 - 1 / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1)))
    term6 = a * (exp(-x) - 1) * (exp(x - xmax) - 1) * ((a * exp(-a * (xmax - x))) / ((exp(-a * x) + 1) * (exp(-a * (xmax - x)) + 1)^2) - (a * exp(-a * x)) / ((exp(-a * x) + 1)^2 * (exp(-a * (xmax - x)) + 1)))
    
    return (term1 - term2 + term3 - term4 + term5 - term6)*per/3
end

function V_mpo(x::Float64, a::Float64, per::Float64, xmax::Float64)::Float64
    sin_term = sin(π * x / per)^2
    exp_term1 = exp(-a * x) + 1.0
    exp_term2 = exp(-a * (xmax - x)) + 1.0

    first_term = sin_term / (exp_term1 * exp_term2)

    exp_term3 = exp(-x)
    exp_term4 = exp(x - xmax)
    second_term = (1.0 - exp_term3) * (exp_term4 - 1.0) * (1.0 - 1.0 / (exp_term1 * exp_term2))

    return first_term + a*second_term
end

function ω(s::Vector{Float64}, state_sizes::Vector{Int}, values::Vector{Vector{Int}}, N::Int, M::Int)
    ω = zeros(N)
    idx = 0
    for i in 1:N
        ω_cont = sum([ 2.0^(q-2)* (1.0+s[q+idx]) for q in 1:state_sizes[i]])
        ω_idx = Int(floor(ω_cont)) # in {0, ..., [log_2 M_i]}
        weight = ω_cont - ω_idx # in (0, 1)

        if ω_idx + 1 >= length(values[i])
            ω[i] = values[i][end]
        else
            if ω_idx + 1 == 1
                ω[i] = weight*values[i][ω_idx+2]
            else
                ω[i] = values[i][ω_idx+1] + ( values[i][ω_idx+2] -values[i][ω_idx+1]) * weight
            end
        end
        idx += state_sizes[i]
    end
    return ω
end

struct Simulation_parameters
    N::Int                                  # Number of assets
    Θ::Matrix{Float64}                      # Connectivity matrix (N by N)
    Jac::SparseMatrixCSC{Float64, Int}      # Sparse Jacobian matrix ω -> s
    M::Int                                  # Number of OPOs
    Budget::Int                             # Budget limit (in integer form)
    Bits_per_asset::Vector{Int}             # Vector (of size N) bits per asset
    p::Float64                              # OPO pump parameter
    α::Float64                              # Target amplitude parameter
    β::Float64                              # Feedback gain parameter
    ω_function::Function                    # Function to get ω from the state s
    values::Vector{Vector{Int}}             # Array of arrays containing all possible values of the assets
end

struct Simulation_results
    solution_time::Float64
    best_configuration::Vector{Int}
    best_obj_value::Float64
end

function load_instance(file_path::String)
    # Loading textual data from file
    parsed_data = JSON.parse(read(file_path, String))

    # Parsing data fields
    N = parsed_data["instance_data"]["N"]
    j_value = parsed_data["instance_data"]["J"]
    J = reshape(hcat(j_value...), (N, N))
    unit = parsed_data["instance_data"]["Unit"]
    max_vals = parsed_data["constraints"]["Upperbox"]
    budget = parsed_data["constraints"]["Budget"]

    # Generating values for parameter struct
    values = [[i for i in 0:val]  for val in max_vals];
    state_sizes = [Int(ceil(log(2, size(val)[1]) )) for val in values]
    M = sum(state_sizes)
    Jak_sparse = get_Jacobian(state_sizes);
    
    return Simulation_parameters(N, J, Jak_sparse, M, budget, state_sizes, 1.9, 1.0, 0.15, ω, values)
end

function simulate(update_rule!::Function, params::Simulation_parameters, Tmax::Float64)
    tspan = (0.0, Tmax)
    
    # initial condition(s)
    u0 = ones(2*params.M +1)
    u0[1:params.M] = 2*rand(params.M)-ones(params.M)
    
    # define problem and run simulation
    prob = ODEProblem(update_rule!, u0, tspan, params)
    sol = solve(prob, Euler(), dt = 1/(5*params.M));
    #sol = solve(prob, Tsit5());

    function clip(s)
        s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
        s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]
        return s_clipped
    end

    s_final = sol.u[end][1:params.M]
    x_final = params.ω_function(clip(s_final), params.Bits_per_asset, params.values, params.N, params.M)
    x_final = round.(Int, x_final)

    return Simulation_results(Tmax, x_final, x_final' * (params.Θ * x_final) )
end

### DEBUG

Binary embedding test

In [ ]:
s_points = [i for i in -1:0.01:1]

m_mat = zeros( (length(s_points), length(s_points)) )

s = -ones(M)
s = [abs(elem) <= 1 ? elem : 0.0 for elem in randn(M)]

for i in 1:length(s_points)
    for j in 1:length(s_points)
        s[6:7] = [s_points[i], s_points[j]]
        m_mat[i, j] = ω(s, state_sizes, values, N, M)[1]
    end
end


In [ ]:
plot(
    heatmap(z=m_mat', x = s_points, y= s_points),
    Layout(xaxis_title="s6", yaxis_title="s7", width= 500, height = 500)
    )

In [ ]:
ωs = [i for i in -0.01:0.0001:3.21]
vals = [i/12 for i in 0:12]
traces = [scatter(x = ωs, y = [DV_mpo(ω, 100.0, 1.0/12, 1.0) for ω in ωs], name="∇V"), 
        #scatter(x = ωs, y = [V_mpo(ω, 1000.0, 1.0/12, 1.0) for ω in ωs], name="V"), 
        scatter(x = ωs, y = [g_ω(1000.0, ω, vals) for ω in ωs], name="g(ω)"), 
        scatter(x = vals, y = zeros(length(vals)), mode="markers", marker_color="red")
        ];
layout = Layout(title="DV_mpo",
                    xaxis_title="ω",
                    yaxis_title="DV_mpo")
plot(traces, layout)

In [ ]:
op_max = 10
elem_nbr = 7

opo_nodes = [i * op_max/elem_nbr for i in 0:elem_nbr-1]
ωs = [i for i in -2.0:0.01:(op_max+2)]

function generate_peeks(nodes, peek_param)
    peek_x = Vector{Float64}()
    peek_y = Vector{Float64}()
    dist = nodes[2] - nodes[1]

    if length(opo_nodes) % 2 == 1
        #push!(peek_x, maximum(nodes) - dist/2)
        #push!(peek_y, dist)
        #push!(peek_x, minimum(nodes) + dist/2)
        #push!(peek_y, dist)


        push!(peek_x, (nodes[Integer(floor(length(nodes)/2))] + nodes[Integer(floor(length(nodes)/2))+1])/2 )
        push!(peek_y, 1.5*dist)
        push!(peek_x, (nodes[1+Integer(floor(length(nodes)/2))] + nodes[Integer(floor(length(nodes)/2))+2])/2 )
        push!(peek_y, 1.5*dist)
    else
        push!(peek_x, (nodes[Integer(length(nodes)/2)] + nodes[Integer(length(nodes)/2)+1])/2 )
        push!(peek_y, 2*dist)
    end
    return (peek_x, peek_y)
end
peek_x, peek_y = generate_peeks(opo_nodes, 0.15)

xs = Vector{Float64}()
ys = Vector{Float64}()

for i in 1:length(opo_nodes)
    push!(xs, opo_nodes[i])
    push!(ys, 0.0)
end

for (p_x, p_y) in zip(peek_x, peek_y)
    push!(xs, p_x)
    push!(ys, p_y)
end

perm=sortperm(xs)
xs = xs[perm]
ys = ys[perm];


p = Lagrange(xs, ys)

traces = [scatter(x = ωs, y = [p(ω)*p(ω) for ω in ωs]), 
scatter(x = opo_nodes, y = zeros(length(opo_nodes)), mode="markers", marker_color="red", name="zeros"),
scatter(x = peek_x, y = peek_y, mode="markers", marker_color="green", name="peeks"),
        ];
layout = Layout(title="Vopo with Lagrange polynomials",
                    xaxis_title="ω",
                    yaxis_title="Vopo = (L(ω; peeks, nodes))²")
plot(traces, layout)

In [ ]:
function chbsv_scaling(points)
        return 2*(points ./ maximum(points)) - ones(size(points))
end

#xs_chbs = chbsv_scaling(xs)

p = Newton(xs, ys)

ωs = [i for i in 0.4:0.001:4.6]

traces = [scatter(x = ωs, y = [p(ω) for ω in ωs]), 
scatter(x = chbsv_scaling(opo_nodes), y = zeros(length(opo_nodes)), mode="markers", marker_color="red", name="zeros"),
scatter(x = chbsv_scaling(peeks), y = ones(length(peeks)), mode="markers", marker_color="green", name="peeks"),
        ];
layout = Layout(title="Vopo with Newton polynomials",
                    xaxis_title="ω",
                    yaxis_title="Vopo")
plot(traces, layout)

In [ ]:
ωs = [i for i in -10:0.001:55]
vals = [0, 10, 20, 30, 40, 50]
traces = [scatter(x = ωs, y = [g_ω(7.0, ω, vals) for ω in ωs]), 
        scatter(x = vals, y = zeros(length(vals)), mode="markers", marker_color="red")
        ];
layout = Layout(title="g(ω)",
                    xaxis_title="ω",
                    yaxis_title="g(ω)")
plot(traces, layout)

### Generating random input data

In [ ]:
N = 24

A = [1 0; 0 0]
B = [0 0; 0 1]


# Random connectivity matrix
J = 2* kron(A, rand(Int(N/2))) *kron(rand(Int(N/2)), A)' + 2 * kron( B, rand(Int(N/2), Int(N/2)) ) - kron( B, ones(Int(N/2), Int(N/2)) ) + 2 * kron(A, rand(Int(N/2), Int(N/2)) ) - kron(A, ones(Int(N/2), Int(N/2)) )
J = 0.5*(J + J')

ω_p = rand(N)
values = Array{Any}(undef, 0)
for i in 1:N
    unit = floor(rand() * 6) + 3
    upper = floor(rand() + unit)
    m_list = 10*[Int(j*unit) for j in 0:upper]
    #push!(values, m_list)
    push!(values, m_list)
end

# Random asset classes
state_sizes = [Int(ceil(log(2, size(val)[1]) )) for val in values]
M = sum(state_sizes)
Jak_sparse = get_Jacobian(state_sizes);

Transforming problem:
$$
H = \boldsymbol{x}^T \boldsymbol{J} \boldsymbol{x} + \boldsymbol{B}^T \boldsymbol{x}
$$

$$\tilde{x}_i = \frac{x_i - x_i^{\text{min}} }{x_i^{\text{max}}}$$

where $x_i \in \left\{ x_i^{\text{min}}, x_i^{\text{min}}+ 1, \dots, x_i^{\text{max}}\right\}$ and $\tilde{x}_i \in \left( 0, 1 \right)$

$$
H = \sum_{ij} J_{ij} \left( \tilde{x}_i \tilde{x}_i  x_i^{\text{max}}  x_j^{\text{max}} +  x_i^{\text{min}}  x_j^{\text{min}} + \tilde{x}_i  x_i^{\text{max}}  x_j^{\text{min}} + \tilde{x}_j  x_j^{\text{max}}  x_i^{\text{min}} \right) + B_i \tilde{x}_i x_i^{\text{max}}  + B_i  x_i^{\text{min}}
$$

$$
H = \boldsymbol{\tilde{x}}^T \boldsymbol{J'} \boldsymbol{\tilde{x}} + \boldsymbol{B'}^T \boldsymbol{\tilde{x}} + 2 \left|\left| \boldsymbol{J} \circ \left( \boldsymbol{\tilde{x}} \circ \boldsymbol{x}^{\text{max}} \cdot (\boldsymbol{x}^{\text{min}})^T \right) \right|\right|_F + \text{const.}
$$

# Update rules

### Update rule 1:

Equations of motion
$$
\frac{d}{dt} \vec{s} = \vec{e}(t) \circ \left[ J^T \partial_{\omega} H(\omega) \right] + \partial V_{opo}(s)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}(t) \circ \left[ \vec{s}(t) \circ \vec{s}(t) - \alpha \vec{1}\right]
$$
where $ \partial H = - \Theta \vec{\omega} (+ \vec{B}) $, and 
$$ \omega_j = \sum_{q=0}^{\left[ log_2 M \right]} 2^{q-2}([s_q] + 1) $$


In [ ]:
function update_rule1!(du, u, params, t)
    Jac_sparse  = params[1]  # Sparse Jacobian matrix ω -> s
    J_ij        = params[2]  # Connectivity matrix
    state_sizes = params[3]  # Vector (of size N) bits per asset
    N           = params[4]  # Number of original variables
    M           = params[5]  # Number of binary variables
    p           = params[6]  # OPO pump parameter
    β           = params[7]  # Feedback gain parameter
    α           = params[8]  # Target amplitude parameter
    ω_function  = params[9]  # Function to get ω from the state s
    values      = params[10] # Array of arrays containing the actual values of the assets
    budget      = params[11] # Budget limit

    # State variables
    e = u[M+1:2M]
    s = u[1:M]
    s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
    s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]


    # N dimensional vector containing the values for the original state
    ω           = ω_function(s_clipped, state_sizes, values, N, M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∇ωH         = J_ij * ω
    ∇sH         = Jac_sparse' * ∇ωH
    ∇sB         = Jac_sparse' * ((sum(ω) - budget) * ones(N))

    # M dimensional vector, containing the OPO dynamics
    ∇V_opo      = (p-1)*s - (s .* s .* s)

    # Normalizing gradient(s)
    ∇sH = normalize(∇sH)
    if norm(∇sB) > 1.0
        normalize!(∇sB)
    end

    # Writing in output vector
    du[1:M] = - e .* (∇sH + ∇sB) + ∇V_opo
    du[M+1:2M] = -β * e .* ( (s .* s) - α * ones(M))
end

### Update rule 1.5:

Equations of motion
$$
\frac{d}{dt} \vec{s} = -\vec{e}_1(t) \circ \left[ J^T \frac{\nabla_{x} H(x)}{\left| \nabla_{x} H(x) \right|} + e_2(t) J^T \frac{\nabla_{x} U(x)}{\left| \nabla_{x} U(x) \right|} \right] - \nabla_s V_{opo}(s)
$$
$$
\frac{d}{dt} \vec{e}_1 = \beta \vec{e}_1(t) \circ \left[ \vec{s}(t) \circ \vec{s}(t) - \alpha \vec{1}\right]
$$
$$
\frac{d}{dt} e_2 = \beta e_2(t) U\left(x(s)\right)
$$
where $ \partial H = - \Theta \vec{x} (+ \vec{B}) $, and $U(s) = \left[ ||x(s)||_{L_1} -K \right]^2$
$$ x_j = \sum_{q=0}^{\left[ log_2 M_j \right]} 2^{q-2}([s_q] + 1) $$


In [ ]:
function update_rule15!_old(du, u, params, t)
    Jac_sparse  = params[1]  # Sparse Jacobian matrix ω -> s
    J_ij        = params[2]  # Connectivity matrix
    state_sizes = params[3]  # Vector (of size N) bits per asset
    N           = params[4]  # Number of original variables
    M           = params[5]  # Number of binary variables
    p           = params[6]  # OPO pump parameter
    β           = params[7]  # Feedback gain parameter
    α           = params[8]  # Target amplitude parameter
    ω_function  = params[9]  # Function to get ω from the state s
    values      = params[10] # Array of arrays containing the actual values of the assets
    budget      = params[11] # Budget limit

    # State variables
    e = u[M+1:2M]
    e2 = u[2M+1]
    s = u[1:M]
    s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
    s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]


    # N dimensional vector containing the values for the original state
    ω           = ω_function(s_clipped, state_sizes, values, N, M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∇ωH         = J_ij * ω
    ∇sH         = Jac_sparse' * ∇ωH
    ∇sB         = Jac_sparse' * ((sum(ω) - budget) * ones(N))

    # M dimensional vector, containing the OPO dynamics
    ∇V_opo      = (p-1)*s - (s .* s .* s)

    # Normalizing gradient(s)
    if norm(∇sH) > 1.0
        normalize!(∇sH)
    end
    if norm(∇sB) > 1.0
        normalize!(∇sB)
    end

    # Writing in output vector
    du[1:M] = - e .* (∇sH + e2 * ∇sB) + ∇V_opo
    du[M+1:2M] = -β * e .* ( (s .* s) - α * ones(M))
    du[2M+1] = β * e2 * (sum(ω) - budget)^2 /budget^2 + 0.01*(1-e2)
end

function update_rule15!(du, u, p, t)
    # State variables
    e = u[p.M+1:2*p.M]
    e2 = u[2*p.M+1]
    s = u[1:p.M]
    s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
    s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]

    # N dimensional vector containing the values for the original state
    ω           = p.ω_function(s_clipped, p.Bits_per_asset, p.values, p.N, p.M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∇ωH         = p.Θ * ω
    ∇sH         = p.Jac' * ∇ωH
    ∇sB         = p.Jac' * ((sum(ω) - p.Budget) * ones(p.N))

    # M dimensional vector, containing the OPO dynamics
    ∇V_opo      = (p.p-1)*s - (s .* s .* s)

    # Normalizing gradient(s)
    if norm(∇sH) > 1.0
        normalize!(∇sH)
    end
    if norm(∇sB) > 1.0
        normalize!(∇sB)
    end

    # Writing in output vector
    du[1:p.M] = - e .* (∇sH + e2 * ∇sB) + ∇V_opo
    du[p.M+1:2*p.M] = -p.β * e .* ( (s .* s) - p.α * ones(p.M))
    du[2*p.M+1] = p.β * e2 * (sum(ω) - p.Budget)^2 /p.Budget^2 
end

### Update rule 2:

Equations of motion:
$$
\frac{d}{dt} \vec{x} = \vec{e}\circ (-\nabla_{x} H) - \mu(t) \nabla_{x} V_{omo} (x; a) - \lambda (t) \left( ||x||_F - K  \right)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}\circ g\left( x;a \right)
$$
where $ \partial H = - \Theta \vec{\omega} (+ \vec{B}) $


In [ ]:
function update_rule2!(du, u, params, t)
    ∂V_omo_function  = params[1] # Sparse Jacobian matrix ω -> s
    J_ij             = params[2] # Connectivity matrix
    N                = params[3] 
    a                = params[4]
    periods          = params[5]
    max_vals         = params[6] 
    β                = params[7] # Feedback gain parameter
    x_zeros          = params[8] # 
    g                = params[9] # Error variable rhs
    ampl             = params[10] 
    budget           = params[11]


    # State variables
    x                = u[1:N]
    e                = u[N+1:2N]

    # N dimensional vector containing the gradient of the quadratic (Ising) potential, including budget constraint
    ∇ωH              = J_ij * x / maximum(max_vals)
    # N dimensional vector, containing the multistable-oscillator dynamics
    ∇V_omo           =  [∂V_omo_function(x[i], a, periods[i], 1.0*x_zeros[i][end]) for i in 1:N]
    # Additional constraints
    ∇Budget          = (sum(x) - budget) * ones(N)

    # Writing in output vector
    du[1:N]          = - e .* ∇ωH - ∇V_omo - ∇Budget
    du[N+1:2N]       = β * e .* [ g(a, x[i], x_zeros[i]) for i in 1:N]
end

In [ ]:
function update_rule2!(du, u, params, t)
    ∂V_omo_function  = params[1] # Sparse Jacobian matrix ω -> s
    J_ij             = params[2] # Connectivity matrix
    N                = params[3] 
    a                = params[4]
    periods          = params[5]
    max_vals         = params[6] 
    β                = params[7] # Feedback gain parameter
    x_zeros          = params[8] # 
    g                = params[9] # Error variable rhs
    ampl             = params[10] 
    budget           = params[11]


    # State variables
    x                = u[1:N]
    e                = u[N+1:2N]

    # N dimensional vector containing the gradient of the quadratic (Ising) potential, including budget constraint
    ∇ωH              = J_ij * x / maximum(max_vals)
    # N dimensional vector, containing the multistable-oscillator dynamics
    ∇V_omo           = [∂V_omo_function(x[i], a, periods[i], 1.0*x_zeros[i][end]) for i in 1:N]
    # Additional constraints
    ∇Budget          = 0.0 * (sum(x) - budget) * ones(N)

    # Writing in output vector
    du[1:N]          = - e .* ∇ωH - ∇V_omo - ∇Budget - 500 * ones(N)
    du[N+1:2N]       = β * e .* [ g(a, x[i], x_zeros[i]) for i in 1:N]
end

### Update rule 3:

Equations of motion:
$$
\frac{d}{dt} s = \vec{e}\circ \partial_{s} H + \partial V_{opo} (s)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}(t) \circ \left[ \vec{s}(t) \circ \vec{s}(t) - \alpha \vec{1}\right]
$$
where 
$$
 \partial_{s_k} H = - 2 \sum_{ij} \Theta_{ij} \omega_i \partial_k \omega_j 
$$


In [ ]:
function update_rule1!(du, u, params, t)
    Jac_sparse  = params[1] # Sparse Jacobian matrix ω -> s
    J_ij        = params[2] # Connectivity matrix
    state_sizes = params[3] # Vector (of size N) bits per asset
    N           = params[4]
    M           = params[5]
    p           = params[6] # OPO pump parameter
    β           = params[7] # Feedback gain parameter
    α           = params[8] # Target amplitude parameter
    get_ω       = params[9] # Function to get ω from the state s
    values      = params[10]# Array of arrays containing the actual values of the assets

    # State variables
    e = u[M+1:2M]
    s = u[1:M]

    # N dimensional vector containing the values for the original state NOT SMOOTH
    ω           = get_ω(s, state_sizes, values, N, M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∂sH         = -J_ij * ω
    # M dimensional vector, containing the OPO dynamics
    ∂V_opo      = (p-1)*s - (s .* s .* s)

    # Writing in output vector
    du[1:M] = e .* (Jac_sparse' * ∇ωH) + ∂V_opo
    du[M+1:2M] = -β * e .* ( (s .* s) - α * ones(M))
end

### Update rule 4:

Equations of motion:
$$
\frac{d}{dt} x = - \partial_{x} H-e \partial_{x} g(x) 
$$
$$
\frac{d}{dt} \vec{e} = g(x)
$$
where 
$$
 \partial_{s_k} H = - 2 \sum_{ij} \Theta_{ij} \omega_i \partial_k \omega_j 
$$


### Update rule AIM

$$
\boldsymbol{x}_{t+1} = \boldsymbol{x}_t + \Delta t \left[ -\alpha \nabla_x F\left( f_{\text{nonlinear}}(\boldsymbol{x}_t) \right) - \beta(t) \boldsymbol{x}_t + \gamma \left( \boldsymbol{x}_t - \boldsymbol{x}_{t-1}\right) \right]
$$

In [ ]:
function update_AIM!(du, u params, t)
    uₘ       = params[1]  # The state at the previous step
    α        = params[2]  # Gradient amplitude
    ∇Ff      = params[3]  # Gradient term with nonlinear clipping
    β        = params[4]  # Annealing schedule
    γ        = params[5]  # Momentum term (γ≥1 is unstable?)


    du = -α ∇Ff(u) - β(t) * u + γ( u - uₘ )
end

function gradient_term(x)

end

function linear_schedule(t, β₀, T)
    return β₀(1-t/T)
end

### Loading data from json file

In [ ]:
parsed_data = JSON.parse(read("QUMO_problems\\N5\\QUMO_N5_ridx0", String))
N = parsed_data["instance_data"]["N"]
j_value = parsed_data["instance_data"]["J"]
J = reshape(hcat(j_value...), (N, N))
unit = parsed_data["instance_data"]["Unit"]

max_vals = parsed_data["constraints"]["Upperbox"]
budget = parsed_data["constraints"]["Budget"] 

periods = ones(N)
x_zeros = [[i for i in 0:val]  for val in max_vals];

# Random asset classes
values = x_zeros
state_sizes = [Int(ceil(log(2, size(val)[1]) )) for val in values]
M = sum(state_sizes)
Jak_sparse = get_Jacobian(state_sizes);

# Simulations

## Simulation 1

In [ ]:
Tmax = 50.0
tspan = (0.0, Tmax)

# Load data here
params = (Jak_sparse, J, state_sizes, N, M, 1.9, 0.15, 1.0, ω, values, budget, Tmax);

# initial condition(s)
u0 = ones(2M)
u0[1:M] = 2*rand(M)-ones(M)

# define problem and run simulation
prob = ODEProblem(update_rule1!, u0, tspan, params)
#sol = solve(prob, Euler(), dt = 0.01);
sol = solve(prob, Tsit5());

## Plotting 1

In [ ]:
function clip(s)
    s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
    s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]
    return s_clipped
end

traces = [scatter(x = sol.t, y = [vec[n] for vec in sol.u], name = "s"*string(n)) for n in 1:M];
layout1 = Layout(title="Softspins",
                   xaxis_title="t",
                   yaxis_title="Value")
p1() = plot(traces, layout1);

auxs = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = "e"*string(m)) for m in M+1:2M];
layout2 = Layout(title="Error variables",
                   xaxis_title="t",
                   yaxis_title="Value")
p2() = plot(auxs, layout2);

energies = [ ω(clip(s_t[1:M]), state_sizes, values, N, M)'*(J *ω(clip(s_t[1:M]), state_sizes, values, N, M)) for s_t in sol.u]
layout3 = Layout(title="Ising energy",
                   xaxis_title="t",
                   yaxis_title="Value")  
p3() = plot([
    scatter(x=sol.t, y= energies, name="Ising Energy"),
    ], layout3); 

budget_vals = [ sum(ω(clip(s_t[1:M]), state_sizes, values, N, M)) for s_t in sol.u]
layout5 = Layout(title="Constraints",
                   xaxis_title="t",
                   yaxis_title="Value")  
p5() = plot([
        scatter(x=sol.t, y= budget_vals, name="Cost")
        scatter(x=sol.t, y= budget*ones(length(sol.t)), name="Budget")
        ], layout5); 

ω_trj = [ ω(clip(s_t[1:M]), state_sizes, values, N, M)' for s_t in sol.u]
traces2 = [scatter(x = sol.t, y = [ω[i] for ω in ω_trj], name = string(i)) for i in 1:N];
layout4 = Layout(title="ω",
                    xaxis_title="t",
                    yaxis_title="Normalized ω")
p4() = plot(traces2, layout4);



## Simulation 1.5

In [ ]:
Tmax = 75.0
tspan = (0.0, Tmax)

# Load data here
params = load_instance("QUMO_problems\\N5\\QUMO_N5_ridx0")
M = params.M
N = params.N
state_sizes = params.Bits_per_asset
values = params.values;

In [ ]:
# initial condition(s)
u0 = ones(2*M +1)
u0[1:M] = 2*rand(M)-ones(M)

# define problem and run simulation
prob = ODEProblem(update_rule15!, u0, tspan, params)
sol = solve(prob, Euler(), dt = 1/(5*M));
#sol = solve(prob, Tsit5());

## Plotting 1.5

In [ ]:
function clip(s)
    s_clipped = [elem < -1.0 ? -1.0 : elem for elem in s]
    s_clipped = [elem > 1.0 ? 1.0 : elem for elem in s_clipped]
    return s_clipped
end

traces = [scatter(x = sol.t, y = [vec[n] for vec in sol.u], name = "s"*string(n)) for n in 1:M];
layout1 = Layout(title="Softspins",
                   xaxis_title="t",
                   yaxis_title="Value")
p1() = plot(traces, layout1);

auxs = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = "e"*string(m-M)) for m in M+1:2M];
layout2 = Layout(title="Error variables",
                   xaxis_title="t",
                   yaxis_title="Value")
p2() = plot(auxs, layout2);

energies = [ ω(clip(s_t[1:M]), state_sizes, values, N, M)'*(params.Θ *ω(clip(s_t[1:M]), state_sizes, values, N, M)) for s_t in sol.u]
layout3 = Layout(title="Ising energy",
                   xaxis_title="t",
                   yaxis_title="Value")  
p3() = plot([
    scatter(x=sol.t, y= energies, name="Ising Energy"),
    ], layout3); 

budget_vals = [ sum(ω(clip(s_t[1:M]), state_sizes, values, N, M)) for s_t in sol.u]
layout5 = Layout(title="Constraints",
                   xaxis_title="t",
                   yaxis_title="Value")  
p5() = plot([
        scatter(x=sol.t, y= budget_vals, name="Cost")
        scatter(x=sol.t, y= params.Budget*ones(length(sol.t)), name="Budget")
        ], layout5); 

ω_trj = [ ω(clip(s_t[1:M]), state_sizes, values, N, M)' for s_t in sol.u]
traces2 = [scatter(x = sol.t, y = [ω[i] for ω in ω_trj], name = string(i)) for i in 1:N];
layout4 = Layout(title="ω",
                    xaxis_title="t",
                    yaxis_title="Normalized ω")
p4() = plot(traces2, layout4);

aux2 = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = "aux"*string(m)) for m in 2M+1:2M+1];
layout6 = Layout(title="Error variables 2",
                   xaxis_title="t",
                   yaxis_title="Value")
p6() = plot(aux2, layout6);



In [ ]:
p = [p1() p2()]

In [ ]:
p = [p4() p6(); p3() p5()]

## Simulation 2

In [ ]:
tspan   = (0.0, 5.0)

function ampl(t, maxval = 0.2, ampl = 1.0)
    if t < maxval
        return 0.0
    else
        return ampl*(t-maxval)^2
    end
end

# Load data here        #a                         #β
params  = (DV_mpo, J, N, 35.0, periods, max_vals, 3.0, x_zeros, g_ω, ampl, budget);


# initial condition(s)
u0 = ones(2N)
#u0[1:N] = [sample(f[2:end]) for f in values] + 2*rand(N) - ones(N)
u0[1:N] = rand(N) .* max_vals
#u0[1:N] = x_best

# define problem and run simulation
prob = ODEProblem(update_rule2!, u0, tspan, params)
sol = solve(prob, Euler(), dt = 0.0005);
#sol = solve(prob, Tsit5());
#sol = solve(prob, RK4());

In [ ]:
traces = [scatter(x = sol.t, y = [vec[n] for vec in sol.u], name = "x"*string(n)) for n in 1:N];
#dummy = scatter(x = [sol.t[end] for i in 1:N], y = x_best, mode="markers", name="best config")
#push!(traces, dummy)
layout1 = Layout(title="x(t)",
                   xaxis_title="t",
                   yaxis_title="x")
p1() = plot(traces, layout1);

auxs = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = "e"*string(m-N)) for m in N+1:2N];
layout2 = Layout(title="Error variables",
                   xaxis_title="t",
                   yaxis_title="Value",
                   yaxis_type="log")
p2() = plot(auxs, layout2);

energies = [ s_t[1:N]'* J *s_t[1:N]/unit for s_t in sol.u]
layout3 = Layout(title="Ising Energy",
                   xaxis_title="t",
                   yaxis_title="Value")   

p3() = plot([
    scatter(x=sol.t, y= energies, name="Ising energy"), 
    #scatter(x=sol.t, y= [H_best/unit for elem in sol.t], name="True ground state")
    ],
 layout3);
p = [p1() p2() p3()]

In [ ]:
x_final = sol.u[2600][1:N]
println(x_final)
println(budget)
sum(x_final)

In [ ]:
x_best = [0.0, 0.0, 772.0, 228.0, 0.0]
H_best = -3549988.6977608744
H_final = (x_final' * J *x_final)


In [ ]:
(H_best - H_final)/H_final

In [ ]:
using Printf
@printf("%.6f\n", 100 * 2.4/abs(H_best/unit))